# Overview of HP search for resnet

In [ ]:
from utils import *

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import glob

from os.path import join
pd.options.display.max_rows = 150

Process logs from HPsearch

In [ ]:
PROJECT_NAME = 'resnet_hp'
PROJECT_DIR = join(MODEL_FOLDER, PROJECT_NAME)

def split_df(df):
    frames = []
    last_split = 0
    last_epoch = 0
    for index, row in df.iterrows():
        if row['epoch'] < last_epoch:
            frames.append(df[last_split:index])
            last_split = index
            last_epoch = 0
        last_epoch = row['epoch']
            
    frames.append(df[last_split:index+1])
    return frames

def avg_change(df, metric):
    data = df.reset_index()
    diff = 0
    diffs = len(data[metric]) - 1
    for i in range(diffs):
        diff += abs(data[metric][i] - data[metric][i+1])
    
    return abs(diff - data[metric].max()) / diffs

trials = dict()
logs = dict()
l=[]
config_paths = glob.glob(join(PROJECT_DIR, 'trial_*_config.json'))
log_paths = glob.glob(join(PROJECT_DIR, 'trial_*_log.csv'))

for config_path in config_paths:
    trial_id = config_path.split('_')[-2]
    with open(config_path, 'r') as f:
        config = json.loads(f.read().replace("'", '"').replace('True', 'true').replace('False', 'false'))
        trials[trial_id] = config
for log_path in log_paths:
    trial_id = log_path.split('_')[-2]
    df = pd.read_csv(log_path)
    m=df['val_cohen_kappa'].max()
    av = 0
    av_diff = 0
    sp = split_df(df)
    for frame in sp:
        av += frame['val_cohen_kappa'].max()
        av_diff = avg_change(frame, 'val_cohen_kappa')
    av = av / len(sp)
    av_diff = av_diff / len(sp)
    
    logs[trial_id] = [len(df.index)/3, av, av_diff]

columns = ['trial_id', 'avg_epochs', 'avg_max_score', 'avg_diff', *trials[trial_id]['values'].keys()]
for key, value in trials.items():
    l.append([key, *logs[key], *value['values'].values()])
    
data = pd.DataFrame(l, columns=columns)
data = data.drop(columns=['opt_beta_1', 'opt_beta_2', 'opt_epsilon', 'bn_momentum'])\
.sort_values(by=['opt_learning_rate', 'batch_size'], ascending=False)
grouped = data.groupby(by=['batch_size', 'opt_learning_rate']).mean()

Display results. Avg_diff represents how much is learning curve "unstable"

In [ ]:
grouped 

In [ ]:
import seaborn as sns
%matplotlib inline

bs= [32, 64, 128, 256]
lr = [0.1, 0.01, 0.001, 0.0001]
lrs = ['0.1', '0.01', '1e-3', '1e-4']
arr = np.zeros((4, 4))

for i in range(4):
    for j in range(4):
         arr[i, j] = grouped.loc[bs[i], lr[j]]['avg_max_score']

sns.set(font_scale=1.4)
ax = sns.heatmap(arr.transpose(), xticklabels=bs, yticklabels=lrs, annot=True)
ax.set(xlabel='Batch size', ylabel='Learning rate')